**Questo notebook contiene:**

**Operazioni di filtering and preprocessing sui dataset**
- p02 + missioni.json
- p03
- p04 
- p05-new + comuni.csv

**Operazioni di join**
- left join pnew5 (anagrafica cig borruso) con p03 (premialità)
- left join il risultato e p04 (quote/deroghe)
- left join del risultato con p01 (associazione cig-cup)
- left join del risultato con p02 (anagrafica cup) 

In [1]:
import pandas as pd
import numpy as np 

In [12]:
# datasets updated version
# pnew = pd.read_csv("p5_prova.csv", sep=';')
# pnew = pd.read_csv("p5tot.csv",sep=";")
pnew = pd.read_csv("./cig_pnrr_anac.csv/cig_pnrr_anac.csv",sep=",")
p03 = pd.read_csv("./datasets/misurepremiali-pnrrpnc_csv.csv",delimiter=';')
p04 = pd.read_csv("./datasets/indicatori-pnrrpnc_csv.csv",delimiter=';')

C:\Users\Alessandra\AppData\Local\Temp\ipykernel_24508\1148437043.py:4: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  pnew = pd.read_csv("./cig_pnrr_anac.csv/cig_pnrr_anac.csv",sep=",")


In [3]:
print(p03['cig'].nunique(), p04['cig'].nunique(), pnew['cig'].nunique())

13545 210648 170481


In [5]:
# datasets
pnew = pd.read_csv("./datasets/pnrr.csv",delimiter=',',encoding = "UTF-8")
p03 = pd.read_excel("./datasets/p06-p03-p04_formattati.xlsx", sheet_name="misurepremiali-pnrrpnc_csv")
p04 = pd.read_excel("./datasets/p06-p03-p04_formattati.xlsx", sheet_name="indicatori-pnrrpnc_csv")

C:\Users\Alessandra\AppData\Local\Temp\ipykernel_10416\3709095954.py:2: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  pnew = pd.read_csv("./datasets/pnrr.csv",delimiter=',',encoding = "UTF-8")


In [17]:
print(p03['cig'].nunique(), p04['cig'].nunique())

11402 178550


In [4]:
p01 = pd.read_csv("./datasets/cup_csv.csv",delimiter=';',encoding = "ISO-8859-1")
p02 = pd.read_excel("./datasets/CUP_PNRR_2023_12_08.xlsx")
comuni = pd.read_csv("./datasets/Elenco-comuni-italiani.csv",delimiter=';',encoding = "ISO-8859-1")

In [5]:
missioni_map = pd.read_json("./missioni_pnrr.json", orient="index")

In [13]:
# preprocessing
pnew = pnew.rename(columns={'cig':'CIG'})
p03 = p03.rename(columns={'cig':'CIG'})
p04 = p04.rename(columns={'cig':'CIG'})

In [14]:
# pre-processing p03
# add colonna 'CATEGORIA'
def conditions(s):
    if s['cod_mis_premiale'] in [1,2,9]:
        return "GENERALE"
    elif s['cod_mis_premiale'] in [3,8,12]:
        return "DISABILI"
    elif s['cod_mis_premiale'] in [4,6,10]:
        return "GENERE"
    elif s['cod_mis_premiale'] in [5,7,11]:
        return "ETÀ"
    else:
        return 0
    
p03['CATEGORIA'] = p03.apply(conditions,axis=1)

In [ ]:
# NON SERVE
# pre-processing p04
# re-encode quote
# preprocessing encoding righe
p04['quota_femminile'] = p04['quota_femminile'].astype("Int32")
p04['quota_giovanile'] = p04['quota_giovanile'].astype("Int32")
p04['quota_femminile'] = p04['quota_femminile'].apply(lambda x: x / 10 ** (len((str(x))) - 2))
p04['quota_giovanile'] = p04['quota_giovanile'].apply(lambda x: x / 10 ** (len((str(x))) - 2))
p04['quota_femminile'] = p04['quota_femminile'].astype("Float64")
p04['quota_giovanile'] = p04['quota_giovanile'].astype("Float64")

In [15]:
# pre-processing p05 pnew
# add colonna 'CLASSE_IMPORTO' 
pd.set_option('display.float_format', lambda x: '%.2f' % x)
def map_importo(importo):
    if importo <= 100000:
        return 'BASSA'
    elif importo <= 1000000:
        return 'MEDIA'
    else:
        return 'ALTA'
    
pnew['CLASSE_IMPORTO'] = pnew['importo_complessivo_gara'].apply(map_importo)
# add colonna 'REGIONI'
prov_reg = comuni[['Denominazione Regione','Denominazione dell\'Unità territoriale sovracomunale \n(valida a fini statistici)']]
prov_reg = prov_reg.rename(columns={'Denominazione dell\'Unità territoriale sovracomunale \n(valida a fini statistici)':'Denominazione sovracomunale','Denominazione Regione':'Regione'})
prov_reg['Denominazione sovracomunale']=prov_reg['Denominazione sovracomunale'].replace(to_replace=r"Valle d'Aosta/Vallée d'Aoste", value='Aosta', regex=False)
prov_reg['Denominazione sovracomunale']=prov_reg['Denominazione sovracomunale'].replace(to_replace=r"Bolzano/Bozen", value='Bolzano',regex=False)
prov_reg['Denominazione sovracomunale']=prov_reg['Denominazione sovracomunale'].str.upper()
prov_reg=prov_reg.drop_duplicates()
pnew=pnew.merge(prov_reg,left_on='provincia', right_on='Denominazione sovracomunale',how='left')
p05_new = pnew[['CIG','importo_complessivo_gara','CLASSE_IMPORTO', 'luogo_istat', 'provincia', 'Denominazione sovracomunale','Regione','data_pubblicazione']]

In [16]:
p05_new

,CIG,importo_complessivo_gara,CLASSE_IMPORTO,luogo_istat,provincia,Regione,Denominazione sovracomunale,data_pubblicazione
0,A0011137E1,21131.90,BASSA,NaN,NaN,NaN,NaN,2023-08-02
1,9923133C64,1626373.58,ALTA,65014.00,SALERNO,Campania,SALERNO,2023-08-07
2,A00574CAF7,172958.95,MEDIA,84041.00,AGRIGENTO,Sicilia,AGRIGENTO,2023-08-18
3,A00114F964,182000.00,MEDIA,82053.00,PALERMO,Sicilia,PALERMO,2023-08-08
4,99556015D7,58442.62,BASSA,68027.00,PESCARA,Abruzzo,PESCARA,2023-08-07
...,...,...,...,...,...,...,...,...
170476,A001EFE42A,149800.00,MEDIA,16024.00,BERGAMO,Lombardia,BERGAMO,2023-08-03
170477,A000A6FCFD,1262.42,BASSA,38007.00,FERRARA,Emilia-Romagna,FERRARA,2023-08-01
170478,99600068F5,101732.97,MEDIA,87049.00,CATANIA,Sicilia,CATANIA,2023-08-14
170479,9946229FCF,668715.94,MEDIA,67039.00,TERAMO,Abruzzo,TERAMO,2023-07-13


In [17]:
# pre-processing p02
# add colonna 'CODICE COMPONENTE', 'CODICE MISSIONE', 'COMPONENTE', 'INVESTIMENTO' 
def map_comp(tematica):
    if tematica.count('-')==1:
        componente = tematica[6:tematica.find('-')-1]
    else:
        componente = tematica[6:tematica.find('I')-4]
    return componente

def map_investimento(tematica):
    # con numeri
    if tematica.count('-')==1:
        investimento = tematica[tematica.find('-')+2:]
    else:
        first = tematica.find('-')+1
        tematica = tematica[first:]
        investimento = tematica[tematica.find('-')+2:]
    return investimento

p02["CODICE_COMPONENTE"] = p02["NOME_TEMATICA"].apply(lambda x: x[:4])
p02["CODICE_MISSIONE"] = p02["NOME_TEMATICA"].apply(lambda x: x[:2])
p02["COMPONENTE"] = p02['NOME_TEMATICA'].apply(map_comp)
p02["INVESTIMENTO"] = p02['NOME_TEMATICA'].apply(map_investimento)
# add colonna 'MISSIONE'
missioni_map.columns = ["MISSIONE"]
p02 = p02.merge(missioni_map,left_on="CODICE_MISSIONE", right_on=missioni_map.index, how='inner')

In [18]:
p02.columns

Index(['CODICE_CUP', 'CODICE_TEMATICA', 'NOME_TEMATICA', 'CODICE_TEMPLATE',
       'DESCRIZIONE_TEMPLATE', 'CODICE_NATURA', 'DESCRIZIONE_NATURA',
       'CODICE_TIPOLOGIA', 'DESCRIZIONE_TIPOLOGIA', 'DESCRIZIONE_AGGREGATA',
       'SOGGETTO_TITOLARE', 'PIVA_SOGGETTO', 'UNITA_ORGANIZZATIVA',
       'STRUMENTO_PROGRAMMAZIONE', 'CODICE_STATO', 'STATO', 'CODICE_REGIONE',
       'REGIONE', 'CODICE_PROVINCIA', 'PROVINCIA', 'CODICE_COMUNE', 'COMUNE',
       'CODICE_SETTORE_INTERVENTO', 'SETTORE_INTERVENTO',
       'CODICE_SOTTOSETTORE_INTERVENTO', 'SOTTOSETTORE_INTERVENTO',
       'CODICE_CATEGORIA_INTERVENTO', 'CATEGORIA_INTERVENTO', 'ANNO_DECISIONE',
       'DATA_GENERAZIONE_COMPLETO', 'COSTO_PROGETTO', 'IMPORTO_FINANZIAMENTO',
       'STATO_PROGETTO', 'TIPO_PROGETTO', 'CODICE_COMPONENTE',
       'CODICE_MISSIONE', 'COMPONENTE', 'INVESTIMENTO', 'MISSIONE'],
      dtype='object')

In [13]:
match_investimenti = p02[["CODICE_TEMATICA","CODICE_MISSIONE",'COMPONENTE',"CODICE_COMPONENTE",'INVESTIMENTO']].drop_duplicates()

Merging procedure
1) left join pnew5 (anagrafica cig borruso) con p03 (premialità)
2) left join il risultato e p04 (quote/deroghe) -> abbiamo tutti i CIG 2O23 con info su premialità e deroghe + importo e regione.

3) left join del risultato con p01 (associazione cig-cup) -> aggiunta una colonna CUP.

4) left join del risultato con p02 (anagrafica cup) -> aggiunta una colonna MISSIONE, COMPONENTE, INVESTIMENTO

In [29]:
p05_new.shape

(170481, 8)

In [31]:
p03

,CIG,cod_mis_premiale,misura_premiale,CATEGORIA
0,9471133234,4,"SI IMPEGNI AD ASSUMERE, OLTRE ALLA SOGLIA MINI...",GENERE
1,94819944FB,2,UTILIZZI O SI IMPEGNI A UTILIZZARE SPECIFICI S...,GENERALE
2,99926503A5,10,ALTRO CRITERIO DI RESPONSABILITÀ SOCIALE NEI C...,GENERE
3,9974793395,2,UTILIZZI O SI IMPEGNI A UTILIZZARE SPECIFICI S...,GENERALE
4,9747626B6F,2,UTILIZZI O SI IMPEGNI A UTILIZZARE SPECIFICI S...,GENERALE
...,...,...,...,...
30286,A0108336FC,8,"ABBIA, NELL'ULTIMO TRIENNIO, RISPETTATO GLI OB...",DISABILI
30287,A0108336FC,10,ALTRO CRITERIO DI RESPONSABILITÀ SOCIALE NEI C...,GENERE
30288,A0108336FC,11,ALTRO CRITERIO DI RESPONSABILITÀ SOCIALE NEI C...,ETÀ
30289,A023A65529,4,"SI IMPEGNI AD ASSUMERE, OLTRE ALLA SOGLIA MINI...",GENERE


In [34]:
# left join pNEW5 e p3
leftnew_3 = p05_new.merge(p03, on='CIG',how='left')
#leftnew_3 = pnew.merge(p03, on='CIG',how='left')
leftnew_3 = leftnew_3.drop_duplicates()

In [35]:
leftnew_3

,CIG,importo_complessivo_gara,CLASSE_IMPORTO,luogo_istat,provincia,Regione,Denominazione sovracomunale,data_pubblicazione,cod_mis_premiale,misura_premiale,CATEGORIA
0,A0011137E1,21131.90,BASSA,NaN,NaN,NaN,NaN,2023-08-02,NaN,NaN,NaN
1,9923133C64,1626373.58,ALTA,65014.00,SALERNO,Campania,SALERNO,2023-08-07,1.00,NEI TRE ANNI ANTECEDENTI LA DATA DI SCADENZA D...,GENERALE
2,A00574CAF7,172958.95,MEDIA,84041.00,AGRIGENTO,Sicilia,AGRIGENTO,2023-08-18,NaN,NaN,NaN
3,A00114F964,182000.00,MEDIA,82053.00,PALERMO,Sicilia,PALERMO,2023-08-08,NaN,NaN,NaN
4,99556015D7,58442.62,BASSA,68027.00,PESCARA,Abruzzo,PESCARA,2023-08-07,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
183566,A001EFE42A,149800.00,MEDIA,16024.00,BERGAMO,Lombardia,BERGAMO,2023-08-03,1.00,NEI TRE ANNI ANTECEDENTI LA DATA DI SCADENZA D...,GENERALE
183567,A000A6FCFD,1262.42,BASSA,38007.00,FERRARA,Emilia-Romagna,FERRARA,2023-08-01,NaN,NaN,NaN
183568,99600068F5,101732.97,MEDIA,87049.00,CATANIA,Sicilia,CATANIA,2023-08-14,NaN,NaN,NaN
183569,9946229FCF,668715.94,MEDIA,67039.00,TERAMO,Abruzzo,TERAMO,2023-07-13,NaN,NaN,NaN


In [22]:
leftnew_3['CIG'].nunique()
leftnew_3[leftnew_3['cod_mis_premiale'].notna()]['data_pubblicazione'].unique()
leftnew_3.to_csv("leftnew_3.csv", index = False, sep=';')

170481

In [36]:
# left join il risultato con p4
leftnew_34 = leftnew_3.merge(p04, on='CIG', how='left')
leftnew_34 = leftnew_34.drop_duplicates()
# leftnew_34.to_csv("leftnew_34.csv", index = False, sep=';')

In [37]:
# left join il risultato con p1
leftnew_341 = leftnew_34.merge(p01, on='CIG', how='left')
leftnew_341 = leftnew_341.drop_duplicates()

In [38]:
# left join il risultato con p2
leftnew_3412 = leftnew_341.merge(p02,how='left',left_on='CUP',right_on='CODICE_CUP')
leftnew_3412 = leftnew_3412.drop_duplicates()

In [39]:
p07 = leftnew_3412

In [40]:
print(p05_new.shape)
print(p07.shape)
p07.columns

(170481, 8)
(273879, 56)


Index(['CIG', 'importo_complessivo_gara', 'CLASSE_IMPORTO', 'luogo_istat',
       'provincia', 'Regione', 'Denominazione sovracomunale',
       'data_pubblicazione', 'cod_mis_premiale', 'misura_premiale',
       'CATEGORIA', 'flag_quote', 'quota_femminile', 'quota_giovanile',
       'cod_mot_deroga', 'mot_deroga', 'CUP', 'CODICE_CUP', 'CODICE_TEMATICA',
       'NOME_TEMATICA', 'CODICE_TEMPLATE', 'DESCRIZIONE_TEMPLATE',
       'CODICE_NATURA', 'DESCRIZIONE_NATURA', 'CODICE_TIPOLOGIA',
       'DESCRIZIONE_TIPOLOGIA', 'DESCRIZIONE_AGGREGATA', 'SOGGETTO_TITOLARE',
       'PIVA_SOGGETTO', 'UNITA_ORGANIZZATIVA', 'STRUMENTO_PROGRAMMAZIONE',
       'CODICE_STATO', 'STATO', 'CODICE_REGIONE', 'REGIONE',
       'CODICE_PROVINCIA', 'PROVINCIA', 'CODICE_COMUNE', 'COMUNE',
       'CODICE_SETTORE_INTERVENTO', 'SETTORE_INTERVENTO',
       'CODICE_SOTTOSETTORE_INTERVENTO', 'SOTTOSETTORE_INTERVENTO',
       'CODICE_CATEGORIA_INTERVENTO', 'CATEGORIA_INTERVENTO', 'ANNO_DECISIONE',
       'DATA_GENERAZIONE

In [43]:
tmp = p07[['CIG', 'importo_complessivo_gara','CLASSE_IMPORTO',
       'cod_mis_premiale', 'misura_premiale', 'CATEGORIA', 'flag_quote',
       'quota_femminile', 'quota_giovanile', 'cod_mot_deroga', 'mot_deroga',
       'CUP', 'REGIONE', 'PROVINCIA', 'COMUNE','ANNO_DECISIONE','data_pubblicazione', 'COSTO_PROGETTO', 'IMPORTO_FINANZIAMENTO', 'CODICE_MISSIONE', 'COMPONENTE',
       'INVESTIMENTO', 'MISSIONE']]

In [83]:
# prima DO NOT RUN
check = tmp.drop_duplicates(subset='CIG')
cig_unici = check.shape[0]
cup_match = check[check['CUP'].notna()].shape[0]
missioni_match = check[check['MISSIONE'].notna()].shape[0]
print('CIG unici',cig_unici)
print('CIG unici con CUP associato',cup_match,':',np.round(cup_match/cig_unici*100,2),'%')
print('CIG unici con MISSIONE associata',missioni_match,':',np.round(missioni_match/cig_unici*100,2),'%')

CIG unici 177312
CIG unici con CUP associato 172767 : 97.44 %
CIG unici con MISSIONE associata 155275 : 87.57 %


In [44]:
# ora
check = tmp.drop_duplicates(subset='CIG')
cig_unici = check.shape[0]
cup_match = check[check['CUP'].notna()].shape[0]
missioni_match = check[check['MISSIONE'].notna()].shape[0]
print('CIG unici',cig_unici)
print('CIG unici con CUP associato',cup_match,':',np.round(cup_match/cig_unici*100,2),'%')
print('CIG unici con MISSIONE associata',missioni_match,':',np.round(missioni_match/cig_unici*100,2),'%')

CIG unici 170481
CIG unici con CUP associato 167858 : 98.46 %
CIG unici con MISSIONE associata 154014 : 90.34 %


In [81]:
# prima DO NOT RUN
tmp = tmp.drop_duplicates()
cig = tmp.shape[0]
cup_match = tmp[tmp['CUP'].notna()].shape[0]
missioni_match = tmp[tmp['MISSIONE'].notna()].shape[0]
print('CIG',cig)
print('CIG con CUP associato',cup_match,':',np.round(cup_match/cig*100,2),'%')
print('CIG con MISSIONE associata',missioni_match,':',np.round(missioni_match/cig*100,2),'%')

CIG 269229
CIG con CUP associato 262643 : 97.55 %
CIG con MISSIONE associata 241274 : 89.62 %


In [45]:
# ora
tmp = tmp.drop_duplicates()
cig = tmp.shape[0]
cup_match = tmp[tmp['CUP'].notna()].shape[0]
missioni_match = tmp[tmp['MISSIONE'].notna()].shape[0]
print('CIG',cig)
print('CIG con CUP associato',cup_match,':',np.round(cup_match/cig*100,2),'%')
print('CIG con MISSIONE associata',missioni_match,':',np.round(missioni_match/cig*100,2),'%')

CIG 273879
CIG con CUP associato 269866 : 98.53 %
CIG con MISSIONE associata 252709 : 92.27 %


In [48]:
# prima DO NOT RUN
tmp.to_csv("tmp_finale.csv", index = False, sep=';')
pd.read_csv("tmp_finale.csv", sep=';')

,CIG,importo_complessivo_gara,CLASSE_IMPORTO,cod_mis_premiale,misura_premiale,CATEGORIA,flag_quote,quota_femminile,quota_giovanile,cod_mot_deroga,...,REGIONE,PROVINCIA,COMUNE,ANNO_DECISIONE,COSTO_PROGETTO,IMPORTO_FINANZIAMENTO,CODICE_MISSIONE,COMPONENTE,INVESTIMENTO,MISSIONE
0,94965511CD,135244.23,MEDIA,NaN,NaN,NaN,S,NaN,NaN,NaN,...,PUGLIA,FOGGIA,NaN,2022.00,369025.00,369025.00,M1,"Digitalizzazione, innovazione e sicurezza nell...",I1.4:Servizi digitali e esperienza dei cittadini,"DIGITALIZZAZIONE, INNOVAZIONE, COMPETITIVITÀ, ..."
1,9719455BFD,18972.95,BASSA,NaN,NaN,NaN,N,NaN,NaN,1.00,...,SICILIA,MESSINA,CESARO',2022.00,23147.00,23147.00,M1,"Digitalizzazione, innovazione e sicurezza nell...",I1.4:Servizi digitali e esperienza dei cittadini,"DIGITALIZZAZIONE, INNOVAZIONE, COMPETITIVITÀ, ..."
2,9576444B94,243287.16,MEDIA,NaN,NaN,NaN,S,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9583207896,287648.76,MEDIA,NaN,NaN,NaN,N,NaN,NaN,8.00,...,PUGLIA,TARANTO,SAN GIORGIO IONICO,2022.00,5500000.00,5500000.00,M6,"Reti di prossimità, strutture e telemedicina p...",I1.1:Case della Comunità e presa in carico del...,SALUTE
4,9469637FA6,4500.00,BASSA,NaN,NaN,NaN,N,NaN,NaN,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269224,A00B0BA7B2,3626.23,BASSA,NaN,NaN,NaN,S,NaN,NaN,NaN,...,TOSCANA,LUCCA,LUCCA,2023.00,4424.00,4424.00,M1,"Digitalizzazione, innovazione e sicurezza nell...",I1.2:Abilitazione al cloud per le PA locali,"DIGITALIZZAZIONE, INNOVAZIONE, COMPETITIVITÀ, ..."
269225,A0173FA7CC,130366.07,MEDIA,NaN,NaN,NaN,N,NaN,NaN,1.00,...,CAMPANIA,NAPOLI,QUALIANO,2022.00,160000.00,160000.00,M2,Tutela del territorio e della risorsa idrica,I2.1:Misure per la gestione del rischio di all...,RIVOLUZIONE VERDE E TRANSIZIONE ECOLOGICA
269226,9998983DCA,21034.21,BASSA,NaN,NaN,NaN,S,NaN,NaN,NaN,...,SICILIA,RAGUSA,MODICA,2022.00,530000.00,530000.00,M6,"Innovazione, ricerca e digitalizzazione del se...",I1.1:Ammodernamento del parco tecnologico e di...,SALUTE
269227,998232070E,8176.00,BASSA,NaN,NaN,NaN,N,NaN,NaN,1.00,...,MARCHE,PESARO E URBINO,PESARO,2022.00,164644.00,164644.00,M4,Potenziamento dell¿offerta dei servizi di istr...,"scuole innovative, nuove aule didattiche e lab...",ISTRUZIONE E RICERCA


In [46]:
# ora
tmp.to_csv("tmp_finale_1.csv", index = False, sep=';')

In [47]:
# ora
tmp_finale1= pd.read_csv("tmp_finale_1.csv", sep=';')

In [48]:
print('CIG unici',tmp_finale1['CIG'].nunique())

CIG unici 170481


Check premialità di genere

In [49]:
check = tmp_finale1[tmp_finale1['CATEGORIA']=='GENERE'].drop_duplicates(subset='CIG')
prem_cig = check.shape[0]
tot_cig = tmp_finale1['CIG'].nunique()
prem_perc = np.round(prem_cig/tot_cig*100,2)

In [50]:
print("Ci sono", prem_cig,"CIG unici con almeno una premialità di genere, su",tot_cig ,"CIG unici.")
print("In percentuale:",prem_perc,'%')

Ci sono 5595 CIG unici con almeno una premialità di genere, su 170481 CIG unici.
In percentuale: 3.28 %


Check deroghe (N e Q)

In [51]:
check1 = tmp_finale1[(tmp_finale1["flag_quote"]=='N')|((tmp_finale1["flag_quote"]=='Q')&(tmp_finale1["quota_femminile"].notna()))]

In [52]:
check1 = check1.drop_duplicates(subset='CIG')

In [53]:
der_cig = check1.shape[0]
der_perc = np.round(der_cig/tot_cig*100,2)

In [54]:
print("Ci sono",der_cig,"CIG unici che derogano la norme rispetto alle quote di assunzioni femminili al 30%, su",tot_cig,"CIG unici.")
print("In percentuale:",der_perc,'%')
print(np.round(100-der_perc,2),"% CIG unici che NON derogano la norme rispetto alle quote di assunzioni femminili al 30%.")

Ci sono 111695 CIG unici che derogano la norme rispetto alle quote di assunzioni femminili al 30%, su 170481 CIG unici.
In percentuale: 65.52 %
34.48 % CIG unici che NON derogano la norme rispetto alle quote di assunzioni femminili al 30%.
